# Dynamic Credit Line Adjustment System  
### CRISP-DM Phase 1: Business & Data Understanding

**Author:** Rudransh Shukla  
**Dataset Size:** ~1.3M Loan Records  
**Objective:** Reduce portfolio Expected Loss using data-driven credit line adjustments

---

This notebook covers:

1. Business Understanding  
2. Risk Framework  
3. Dataset Overview  
4. Target Definition  
5. Data Quality Assessment  
6. Leakage Identification  
7. Data Limitations  

No modeling is performed in this notebook.


## 1️. Business Understanding

Financial institutions extend revolving credit to customers with varying levels of creditworthiness.

Static credit limits can expose lenders to excessive risk if high-risk borrowers are granted high exposure.

The objective of this project is to:

- Predict Probability of Default (PD)
- Segment customers into risk buckets
- Dynamically adjust credit limits
- Reduce overall portfolio Expected Loss (EL)

Rather than rejecting customers, the strategy focuses on optimizing exposure based on risk.


## 2️. Credit Risk Framework

Credit risk is commonly modeled using three components:

\[
Expected Loss = PD * LGD * EAD
\]

Where:

- **PD (Probability of Default):** Likelihood that a borrower defaults.
- **LGD (Loss Given Default):** Percentage of exposure lost if default occurs.
- **EAD (Exposure at Default):** Total exposure at the time of default.

In this project:

- PD will be modeled using Logistic Regression.
- LGD is assumed constant at 60% for simulation purposes.
- EAD is approximated using revolving credit limit.

The business goal is to reduce portfolio Expected Loss by adjusting credit limits based on predicted PD.


## 3️. Project Objective

The project follows the CRISP-DM methodology.

Phase 1 (This Notebook):
- Understand business objective
- Explore dataset structure
- Define target variable
- Identify data leakage
- Assess data quality

Subsequent Phases:
- Exploratory Data Analysis
- PD Model Development
- Model Evaluation
- Credit Limit Adjustment Simulation
- Portfolio Expected Loss Impact Analysis


In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("../data/raw/accepted_2007_to_2018Q4.csv" , low_memory= False)
df.to_parquet("loan_data.parquet")

In [3]:
df = pd.read_parquet("../data/parquet/loan_data.parquet")

## 4️. Dataset Overview

The dataset consists of approximately 1.3 million historical loan records.

Each row represents a single loan issued to a borrower.

The dataset contains:

- Loan characteristics (loan amount, term, interest rate)
- Borrower attributes (income, credit history, employment length)
- Revolving credit information
- Loan status outcomes

The goal is to use historical loan performance to predict default risk.


In [ ]:
df.head()

In [ ]:
df.info()

In [4]:
df_cols = df.columns.tolist()

In [5]:
selected_features = [
    # Loan / Product Context
    "loan_amnt", "term", "int_rate", "installment", "purpose",

    # Credit Utilization & Exposure
    "revol_util", "revol_bal", "bc_util", "all_util",
    "total_rev_hi_lim", "bc_open_to_buy", "max_bal_bc",
    "total_bc_limit", "total_bal_ex_mort",

    # Credit Capacity
    "tot_hi_cred_lim", "total_il_high_credit_limit",
    "total_bal_il", "avg_cur_bal",

    # Credit History & FICO (origination only)
    "fico_range_low", "fico_range_high",
    "earliest_cr_line",

    # Time / Aging
    "issue_d", "mo_sin_old_rev_tl_op", "mo_sin_old_il_acct",

    # Recent Credit Behavior
    "inq_last_6mths", "inq_last_12m",
    "acc_open_past_24mths", "mths_since_recent_inq",
    "mths_since_recent_bc", "mths_since_recent_revol_delinq",

    # Account Structure
    "open_acc", "total_acc", "num_rev_accts",
    "num_rev_tl_bal_gt_0", "num_actv_rev_tl",
    "num_bc_tl", "num_bc_sats", "num_sats",

    # Delinquency & Risk
    "delinq_2yrs", "acc_now_delinq", "delinq_amnt",
    "pub_rec", "pub_rec_bankruptcies",
    "collections_12_mths_ex_med", "tax_liens",

    # Income & Stability
    "annual_inc", "verification_status",
    "emp_length", "home_ownership",

    # Geographic (proxy)
    "addr_state",

    # Target + status
    "loan_status"
]

In [6]:
df['loan_status'].unique()

array(['Fully Paid', 'Current', 'Charged Off', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', 'Default', None,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [7]:
df = df[df['loan_status'].isin(['Fully Paid' , 'Charged Off'])]

### Why we exclude loans with status: Current, Late, In Grace Period

To train a supervised model, the target outcome must be **final and known**.  
The objective of this model is to learn patterns that **lead to loan default**.

- For **Fully Paid** and **Charged Off** loans, the outcome is resolved and labels are reliable.
- Loans marked as **Current**, **Late**, or **In Grace Period** are still ongoing.
- Their final outcome is unknown: a current loan may default later, and a late loan may recover.

Including such loans would introduce **label uncertainty**, leading to incorrect learning and unreliable predictions.

Therefore, these loan statuses are excluded during training and are used **only at inference time** to assess risk and support credit-line decisions.


In [8]:
df = df[selected_features]

In [9]:
df['default'] = (df['loan_status'] == 'Charged Off').astype(int)

## 5️. Target Variable Definition

The target variable for this project is **default**.

Default is defined as:

- Loans marked as **Charged Off** → Default = 1
- Loans marked as **Fully Paid** → Default = 0

This binary target allows us to model Probability of Default (PD).


since we will be training a model on the data it is very important to change the ***non integer*** columns to *integer* colums

In [11]:
x = df.select_dtypes(include = (object))

In [12]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1345310 entries, 0 to 2260697
Data columns (total 9 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   term                 1345310 non-null  object
 1   purpose              1345310 non-null  object
 2   earliest_cr_line     1345310 non-null  object
 3   issue_d              1345310 non-null  object
 4   verification_status  1345310 non-null  object
 5   emp_length           1266799 non-null  object
 6   home_ownership       1345310 non-null  object
 7   addr_state           1345310 non-null  object
 8   loan_status          1345310 non-null  object
dtypes: object(9)
memory usage: 102.6+ MB


In [ ]:
df['default'].value_counts(normalize=True)

## 6️. Class Distribution

The dataset exhibits class imbalance, with a significantly smaller proportion of defaulted loans compared to non-defaulted loans.

Class imbalance is common in credit risk modeling and has implications for:

- Model evaluation metrics
- Threshold selection
- Calibration quality

A stratified train-test split will be used in later phases to preserve class proportions.


## 7️. Initial Data Cleaning & Preparation

Before performing exploratory analysis or modeling, basic data cleaning steps were applied to ensure consistency and usability.

These transformations include:

- Correcting data types
- Removing formatting inconsistencies
- Engineering foundational features
- Standardizing numeric representations

These steps ensure the dataset is structurally ready for analysis while preserving modeling integrity.


In [13]:
df.loc[:, "issue_d"] = pd.to_datetime(
    df["issue_d"],
    format="%b-%Y"
)


In [14]:
df.loc[:, "earliest_cr_line"] = pd.to_datetime(
    df["earliest_cr_line"],
    format="%b-%Y"
)


In [15]:
df.loc[:,'term'] = df['term'].str.strip()

In [16]:
df['term'].unique()

array(['36 months', '60 months'], dtype=object)

In [17]:
df.loc[:,'term'] = df['term'].map({
    '36 months' : 36,
    '60 months' : 60
})

In [18]:
df['term'] = df.term.astype(int)

In [22]:
df['issue_d'] = pd.to_datetime(df['issue_d'])

In [23]:
df["earliest_cr_line"] = pd.to_datetime(
    df["earliest_cr_line"],
    format="%Y-%m-%d"
)


In [24]:
df['credit_age'] = ((df['issue_d'] - df['earliest_cr_line']).dt.days/30).clip(lower=0)

df = df.drop(columns=["earliest_cr_line"])

now ***verification status***

In [25]:
df.verification_status.value_counts()

verification_status
Source Verified    521273
Verified           418336
Not Verified       405701
Name: count, dtype: int64

In [26]:
df["verification_status"] = df["verification_status"].str.strip()

In [30]:
df.emp_length.value_counts()

emp_length
10    442199
2     121743
0     108061
3     107597
1      88494
5      84154
4      80556
6      62733
8      60701
7      59624
9      50937
Name: count, dtype: int64

In [28]:
df['emp_length'] = df['emp_length'].str.strip()

In [32]:
df['emp_length'] = (df['emp_length'].str.replace('+ years','')
                   .str.replace('< 1 year','0')
                    .str.replace(' years',' ')
                    .str.replace(' year',' ')
                   )


In [33]:
df['emp_length'] = pd.to_numeric(df['emp_length'])

In [34]:
df["home_ownership"] = df["home_ownership"].str.strip()

In [35]:
df["addr_state"] = df["addr_state"].str.strip()

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1345310 entries, 0 to 2260697
Data columns (total 52 columns):
 #   Column                          Non-Null Count    Dtype         
---  ------                          --------------    -----         
 0   loan_amnt                       1345310 non-null  float64       
 1   term                            1345310 non-null  int32         
 2   int_rate                        1345310 non-null  float64       
 3   installment                     1345310 non-null  float64       
 4   purpose                         1345310 non-null  object        
 5   revol_util                      1344453 non-null  float64       
 6   revol_bal                       1345310 non-null  float64       
 7   bc_util                         1283398 non-null  float64       
 8   all_util                        537545 non-null   float64       
 9   total_rev_hi_lim                1277783 non-null  float64       
 10  bc_open_to_buy                  1284167 non-nul

In [37]:
df.drop('issue_d',axis = 1 ,inplace=True)

check for any ***leakage columns***

In [38]:
leakage_cols = [
    "recoveries",
    "collection_recovery_fee",
    "last_pymnt_amnt",
    "total_rec_prncp",
    "total_rec_int",
    "total_pymnt",
    "out_prncp"
]

[col for col in leakage_cols if col in df.columns]


[]

In [39]:
df.select_dtypes(include=["datetime"])

""
0
1
2
4
5
...
2260688
2260690
2260691
2260692


check for any infinite values

In [40]:
import numpy as np
np.isinf(df.select_dtypes(include=np.number)).sum().sum()

0

## 8️. Data Readiness Check

At this stage:

- Target variable is defined
- Data leakage variables are removed
- Missing values are assessed
- Core features are cleaned and standardized
- Foundational feature engineering is complete

The dataset is now structurally prepared for Exploratory Data Analysis in the next phase.


In [42]:
import os

In [48]:
os.makedirs("../data/processed", exist_ok=True)

In [49]:
df.to_parquet("../data/processed/phase2_clean.parquet", index=False)